In [1]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SparkSession

val conf = new SparkConf().setAppName("response_count").setMaster("local[2]")
    val sc = new SparkContext(conf)
    val spark = SparkSession.builder.getOrCreate

    //convert raw rating
    var train_raw = sc.textFile("./Data/train_review.csv")
    var test_raw = sc.textFile("./Data/test_review.csv")
    val train_header = train_raw.first()
    train_raw = train_raw.filter(row => row != train_header)
    val test_header = test_raw.first()
    test_raw = test_raw.filter(row => row != test_header)
    val n = test_raw.count()

    var train_rating = train_raw.map(_.split(','))
    var test_rating = test_raw.map(_.split(','))
    var all_rating = train_rating.union(test_rating)
    var user = all_rating.map(x => (x(0), 1)).reduceByKey((a, b) => a + b)
    val user_encode = user.map(x => x._1).zipWithUniqueId().collect().toMap
    var train_rating_1 = train_rating.map(x => (user_encode(x(0)).toInt, x(1), x(2)))
    var test_rating_1 = test_rating.map(x => (user_encode(x(0)).toInt, x(1), x(2)))

    var product = all_rating.map(x => (x(1), 1)).reduceByKey((a, b) => a + b)
    val product_encode = product.map(x => x._1).zipWithUniqueId().collect().toMap
    var train_rating_2 = train_rating_1.map(x => (x._1, product_encode(x._2).toInt, x._3.toDouble))
    var test_rating_2 = test_rating_1.map(x => (x._1, product_encode(x._2).toInt, x._3.toDouble))
    var all_rating_encode = train_rating_2++test_rating_2

conf = org.apache.spark.SparkConf@1198be13
sc = org.apache.spark.SparkContext@1e3d63e9
spark = org.apache.spark.sql.SparkSession@18c1bf55
train_raw = MapPartitionsRDD[4] at filter at <console>:38
test_raw = MapPartitionsRDD[5] at filter at <console>:40
train_header = user_id,business_id,stars
train_raw = MapPartitionsRDD[4] at filter at <console>:38
test_header = user_id,business_id,stars
test_raw = MapPartitionsRDD[5] at filter at <console>:40
n = 4...


45236

In [2]:
    var user_key = train_rating_2.map(x=>(x._1,x))// (user, (user, item, rate))
//    var user_join = user_key.join(user_key).map(x=>(x._1,((x._2._1._2,x._2._1._3),(x._2._2._2,x._2._2._3))))
    var user_join = user_key.join(user_key).filter(x=>x._2._1._2<=x._2._2._2).map(x=>(x._1,((x._2._1._2,x._2._1._3),(x._2._2._2,x._2._2._3))))
    // (user, ((i1,r1),(i2,r2)))
    var item_pair = user_join.map(x=>((x._2._1._1,x._2._2._1), (x._1, x._2._1._2,x._2._2._2))) //  ((i1,i2),(u,r1,r2))
    var item_pair_count = item_pair.groupByKey().map(x=>(x._1, x._2.size)).filter(x=>x._2>100).count

Waiting for a Spark session to start...

user_key = MapPartitionsRDD[22] at map at <console>:41
user_join = MapPartitionsRDD[27] at map at <console>:43
item_pair = MapPartitionsRDD[28] at map at <console>:45
item_pair_count = 433


433

In [ ]:
print(item_pair_count)